In [70]:
import os
import pandas as pd
import json
import torch
import numpy as np

import seaborn as sns
sns.set()

log_path = "./logs_09_march"    

In [3]:

def load_row(mesh_name, exp_path):

    # load params.json
    with open(os.path.join(exp_path, "params.json")) as f:
        params = json.load(f)

    # load summary.json
    with open(os.path.join(exp_path, "summary.json")) as f:
        summary = json.load(f)
    # load delta
    ckpt_path = os.path.join(exp_path, "checkpoints", f"{mesh_name}.ckpt")
    delta = torch.load(ckpt_path)

    return params, summary, delta



In [4]:
data = []

for mesh in os.listdir(log_path):
    mesh_path = os.path.join(log_path, mesh, "pgd_attack")
    
    for exp in os.listdir(mesh_path):
        exp_path = os.path.join(mesh_path, exp, "version_0")
        params, summary, delta = load_row(mesh, exp_path)
        n_pixels = (delta != 0.).sum().to(dtype=torch.float32) / delta.numel()
        n_pixels = float(n_pixels)
        data.append({
            "name": mesh,
            **params,
            **summary,
            "n_pixels": n_pixels
        })

df = pd.DataFrame(data)

        

In [5]:
# filter used hparams

alpha = [0.01]
df2 = df[df['alpha'].isin(alpha)]

tex_scale = [0.33]
df2 = df2[df2['texture_rescale'].isin(tex_scale)]


df = df2
df

,name,eps,alpha,model,saliency,texture_rescale,saliency_threshold,pytorch_no_attack,pytorch_attack,sailenv_no_attack,sailenv_attack,n_pixels
1,candle,0.05,0.01,inception,False,0.33,NaN,1.000000,0.000000,0.866667,0.766667,0.469931
2,candle,0.05,0.01,inception,True,0.33,0.05,1.000000,0.066667,0.900000,0.733333,0.214156
3,candle,0.05,0.01,inception,True,0.33,0.20,1.000000,1.000000,0.900000,0.983333,0.010806
4,candle,0.05,0.01,mobilenet,False,0.33,NaN,0.566667,0.000000,0.583333,0.166667,0.354356
5,candle,0.05,0.01,mobilenet,True,0.33,0.05,0.566667,0.000000,0.583333,0.166667,0.319208
...,...,...,...,...,...,...,...,...,...,...,...,...
287,toilet,0.50,0.01,inception,True,0.33,0.05,0.600000,0.050000,0.016667,0.016667,0.085846
288,toilet,0.50,0.01,inception,True,0.33,0.20,0.600000,0.016667,0.016667,0.000000,0.022172
289,toilet,0.50,0.01,mobilenet,False,0.33,NaN,0.600000,0.000000,0.250000,0.000000,0.095210
290,toilet,0.50,0.01,mobilenet,True,0.33,0.05,0.600000,0.000000,0.250000,0.000000,0.084698


In [200]:
# add accuracy drop

df['sailenv_acc_drop'] = df['sailenv_no_attack'] - df['sailenv_attack']
df['pytorch_acc_drop'] = df['pytorch_no_attack'] - df['pytorch_attack']

df

,name,eps,alpha,model,saliency,texture_rescale,saliency_threshold,pytorch_no_attack,pytorch_attack,sailenv_no_attack,sailenv_attack,n_pixels,acc_drop,sailenv_acc_drop,pytorch_acc_drop
1,candle,0.05,0.01,inception,False,0.33,NaN,1.000000,0.000000,0.866667,0.766667,0.469931,0.100000,0.100000,1.000000
2,candle,0.05,0.01,inception,True,0.33,0.05,1.000000,0.066667,0.900000,0.733333,0.214156,0.166667,0.166667,0.933333
3,candle,0.05,0.01,inception,True,0.33,0.20,1.000000,1.000000,0.900000,0.983333,0.010806,-0.083333,-0.083333,0.000000
4,candle,0.05,0.01,mobilenet,False,0.33,NaN,0.566667,0.000000,0.583333,0.166667,0.354356,0.416667,0.416667,0.566667
5,candle,0.05,0.01,mobilenet,True,0.33,0.05,0.566667,0.000000,0.583333,0.166667,0.319208,0.416667,0.416667,0.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,toilet,0.50,0.01,inception,True,0.33,0.05,0.600000,0.050000,0.016667,0.016667,0.085846,0.000000,0.000000,0.550000
288,toilet,0.50,0.01,inception,True,0.33,0.20,0.600000,0.016667,0.016667,0.000000,0.022172,0.016667,0.016667,0.583333
289,toilet,0.50,0.01,mobilenet,False,0.33,NaN,0.600000,0.000000,0.250000,0.000000,0.095210,0.250000,0.250000,0.600000
290,toilet,0.50,0.01,mobilenet,True,0.33,0.05,0.600000,0.000000,0.250000,0.000000,0.084698,0.250000,0.250000,0.600000


In [214]:
grp_by_eps = df.groupby(["name", "eps", "model"])

loc_of_maxes = grp_by_eps.idxmax()['pytorch_acc_drop']

best_exps = df.loc[loc_of_maxes]
best_exps

,name,eps,alpha,model,saliency,texture_rescale,saliency_threshold,pytorch_no_attack,pytorch_attack,sailenv_no_attack,sailenv_attack,n_pixels,acc_drop,sailenv_acc_drop,pytorch_acc_drop
1,candle,0.05,0.01,inception,False,0.33,NaN,1.000000,0.000000,0.866667,0.766667,0.469931,0.100000,0.100000,1.000000
4,candle,0.05,0.01,mobilenet,False,0.33,NaN,0.566667,0.000000,0.583333,0.166667,0.354356,0.416667,0.416667,0.566667
26,candle,0.10,0.01,inception,False,0.33,NaN,1.000000,0.000000,0.875000,0.525000,0.429315,0.350000,0.350000,1.000000
35,candle,0.10,0.01,mobilenet,False,0.33,NaN,0.525000,0.000000,0.583333,0.108333,0.286458,0.475000,0.475000,0.525000
80,candle,0.50,0.01,inception,False,0.33,NaN,1.000000,0.000000,0.875000,0.566667,0.493906,0.308333,0.308333,1.000000
89,candle,0.50,0.01,mobilenet,False,0.33,NaN,0.525000,0.000000,0.583333,0.116667,0.286765,0.466667,0.466667,0.525000
115,lamp_floor,0.05,0.01,inception,False,0.33,NaN,0.700000,0.000000,0.766667,1.000000,0.219144,-0.233333,-0.233333,0.700000
119,lamp_floor,0.05,0.01,mobilenet,True,0.33,0.05,0.833333,0.000000,1.000000,0.983333,0.160525,0.016667,0.016667,0.833333
121,lamp_floor,0.10,0.01,inception,False,0.33,NaN,0.700000,0.000000,0.766667,1.000000,0.219338,-0.233333,-0.233333,0.700000
125,lamp_floor,0.10,0.01,mobilenet,True,0.33,0.05,0.833333,0.000000,1.000000,0.966667,0.144770,0.033333,0.033333,0.833333


In [202]:
inception_df = df[df['model'] == 'inception']
pivoted = df.pivot(index=['model','eps',"saliency_threshold"], columns="name", values=["sailenv_acc_drop", "pytorch_acc_drop", 'n_pixels'])

pivoted

sailenv_acc_drop                       \
name                                        candle lamp_floor   plunger   
model     eps  saliency_threshold                                         
inception 0.05 NaN                        0.100000  -0.233333  0.000000   
               0.05                       0.166667  -0.216667  0.000000   
               0.20                      -0.083333  -0.216667  0.000000   
          0.10 NaN                        0.350000  -0.233333  0.000000   
               0.05                       0.433333  -0.216667  0.000000   
               0.20                      -0.091667  -0.216667  0.000000   
          0.50 NaN                        0.308333  -0.233333  0.000000   
               0.05                       0.358333  -0.216667  0.000000   
               0.20                      -0.116667  -0.216667  0.000000   
mobilenet 0.05 NaN                        0.416667   0.000000  0.250000   
               0.05                       0.416667   0.016667  0.266667   
               0.20                       0.300000   0.000000  0.283333   
          0.10 NaN                        0.475000   0.000000  0.250000   
               0.05                       0.400000   0.033333  0.233333   
               0.20                       0.300000   0.000000  0.300000   
          0.50 NaN                        0.466667   0.000000  0.233333   
               0.05                       0.408333   0.000000  0.300000   
               0.20                       0.258333   0.000000  0.233333   

                                                                       \
name                              remote_controller table_living_room   
model     eps  saliency_threshold                                       
inception 0.05 NaN                         0.233333          0.066667   
               0.05                        0.066667          0.200000   
               0.20                        0.000000          0.000000   
          0.10 NaN                         0.583333          0.200000   
               0.05                        0.400000          0.366667   
               0.20                        0.033333          0.000000   
          0.50 NaN                         0.850000          0.250000   
               0.05                             NaN          0.450000   
               0.20                        0.033333          0.083333   
mobilenet 0.05 NaN                         0.366667          0.150000   
               0.05                        0.116667          0.133333   
               0.20                        0.066667          0.000000   
          0.10 NaN                         0.633333          0.166667   
               0.05                        0.166667          0.166667   
               0.20                        0.100000          0.000000   
          0.50 NaN                         0.533333          0.166667   
               0.05                        0.183333          0.166667   
               0.20                        0.216667         -0.033333   

                                                          pytorch_acc_drop  \
name                              tennis_racket    toilet           candle   
model     eps  saliency_threshold                                            
inception 0.05 NaN                     0.000000  0.033333         1.000000   
               0.05                    0.000000  0.000000         0.933333   
               0.20                    0.000000 -0.016667         0.000000   
          0.10 NaN                     0.000000  0.033333         1.000000   
               0.05                    0.000000  0.000000         0.991667   
               0.20                    0.000000 -0.033333         0.150000   
          0.50 NaN                     0.000000  0.033333         1.000000   
               0.05                    0.000000  0.000000         0.925000   
               0.20                    0.000000  0.016667         

In [206]:
# print sailenv_acc_drop table
acc_drop_df = pivoted['sailenv_acc_drop']
acc_drop_df['avg'] = acc_drop_df.mean(numeric_only=True, axis=1)
print(acc_drop_df.round(2).to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
          &      & name &  candle &  lamp\_floor &  plunger &  remote\_controller &  table\_living\_room &  tennis\_racket &  toilet &   avg \\
model & eps & saliency\_threshold &         &             &          &                    &                    &                &         &       \\
\midrule
inception & 0.05 & NaN &    0.10 &       -0.23 &     0.00 &               0.23 &               0.07 &           0.00 &    0.03 &  0.03 \\
          &      & 0.05 &    0.17 &       -0.22 &     0.00 &               0.07 &               0.20 &           0.00 &    0.00 &  0.03 \\
          &      & 0.20 &   -0.08 &       -0.22 &     0.00 &               0.00 &               0.00 &           0.00 &   -0.02 & -0.05 \\
          & 0.10 & NaN &    0.35 &       -0.23 &     0.00 &               0.58 &               0.20 &           0.00 &    0.03 &  0.13 \\
          &      & 0.05 &    0.43 &       -0.22 &     0.00 &               0.40 &               0.37 &    

In [208]:
# print acc_drop table
acc_drop_df = pivoted['pytorch_acc_drop']
acc_drop_df['avg'] = acc_drop_df.mean(numeric_only=True, axis=1)
print(acc_drop_df.round(2).to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
          &      & name &  candle &  lamp\_floor &  plunger &  remote\_controller &  table\_living\_room &  tennis\_racket &  toilet &   avg \\
model & eps & saliency\_threshold &         &             &          &                    &                    &                &         &       \\
\midrule
inception & 0.05 & NaN &    1.00 &        0.70 &     0.88 &               0.53 &               0.57 &           0.00 &    0.50 &  0.60 \\
          &      & 0.05 &    0.93 &        0.70 &     0.88 &               0.35 &               0.48 &           0.00 &    0.53 &  0.55 \\
          &      & 0.20 &    0.00 &        0.05 &     0.72 &               0.05 &               0.05 &           0.00 &    0.52 &  0.20 \\
          & 0.10 & NaN &    1.00 &        0.70 &     0.85 &               0.73 &               0.62 &           0.12 &    0.50 &  0.65 \\
          &      & 0.05 &    0.99 &        0.70 &     0.88 &               0.78 &               0.55 &    

In [198]:
# print n_pixels table
npixels_df = pivoted['n_pixels']
npixels_df['avg'] = npixels_df.mean(numeric_only=True, axis=1)
print(npixels_df.round(2).to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
          &      & name &  candle &  lamp\_floor &  plunger &  remote\_controller &  table\_living\_room &  tennis\_racket &  toilet &   avg \\
model & eps & saliency\_threshold &         &             &          &                    &                    &                &         &       \\
\midrule
inception & 0.05 & NaN &    0.47 &        0.22 &     0.40 &               0.44 &               0.42 &           0.24 &    0.10 &  0.33 \\
          &      & 0.05 &    0.21 &        0.20 &     0.38 &               0.31 &               0.29 &           0.17 &    0.09 &  0.24 \\
          &      & 0.20 &    0.01 &        0.04 &     0.17 &               0.04 &               0.01 &           0.03 &    0.02 &  0.05 \\
          & 0.10 & NaN &    0.43 &        0.22 &     0.40 &               0.40 &               0.39 &           0.22 &    0.10 &  0.31 \\
          &      & 0.05 &    0.26 &        0.20 &     0.38 &               0.36 &               0.28 &    

In [188]:
pivoted

acc_drop                       \
name                                 candle lamp_floor   plunger   
model     eps  saliency_threshold                                  
inception 0.05 NaN                 0.100000  -0.233333  0.000000   
               0.05                0.166667  -0.216667  0.000000   
               0.20               -0.083333  -0.216667  0.000000   
          0.10 NaN                 0.350000  -0.233333  0.000000   
               0.05                0.433333  -0.216667  0.000000   
               0.20               -0.091667  -0.216667  0.000000   
          0.50 NaN                 0.308333  -0.233333  0.000000   
               0.05                0.358333  -0.216667  0.000000   
               0.20               -0.116667  -0.216667  0.000000   
mobilenet 0.05 NaN                 0.416667   0.000000  0.250000   
               0.05                0.416667   0.016667  0.266667   
               0.20                0.300000   0.000000  0.283333   
          0.10 NaN                 0.475000   0.000000  0.250000   
               0.05                0.400000   0.033333  0.233333   
               0.20                0.300000   0.000000  0.300000   
          0.50 NaN                 0.466667   0.000000  0.233333   
               0.05                0.408333   0.000000  0.300000   
               0.20                0.258333   0.000000  0.233333   

                                                                       \
name                              remote_controller table_living_room   
model     eps  saliency_threshold                                       
inception 0.05 NaN                         0.233333          0.066667   
               0.05                        0.066667          0.200000   
               0.20                        0.000000          0.000000   
          0.10 NaN                         0.583333          0.200000   
               0.05                        0.400000          0.366667   
               0.20                        0.033333          0.000000   
          0.50 NaN                         0.850000          0.250000   
               0.05                             NaN          0.450000   
               0.20                        0.033333          0.083333   
mobilenet 0.05 NaN                         0.366667          0.150000   
               0.05                        0.116667          0.133333   
               0.20                        0.066667          0.000000   
          0.10 NaN                         0.633333          0.166667   
               0.05                        0.166667          0.166667   
               0.20                        0.100000          0.000000   
          0.50 NaN                         0.533333          0.166667   
               0.05                        0.183333          0.166667   
               0.20                        0.216667         -0.033333   

                                                           n_pixels  \
name                              tennis_racket    toilet    candle   
model     eps  saliency_threshold                                     
inception 0.05 NaN                     0.000000  0.033333  0.469931   
               0.05                    0.000000  0.000000  0.214156   
               0.20                    0.000000 -0.016667  0.010806   
          0.10 NaN                     0.000000  0.033333  0.429315   
               0.05                    0.000000  0.000000  0.264243   
               0.20                    0.000000 -0.033333  0.030919   
          0.50 NaN                     0.000000  0.033333  0.493906   
               0.05                    0.000000  0.000000  0.239985   
               0.20                    0.000000  0.016667  0.029171   
mobilenet 0.05 NaN                     0.250000  0.250000  0.354356   
               0.05                    0.266667  0.250000  0.319208   
               0.20                    0.000000 -0.083333  0.147581   
       